## 1.Preparación de entorno

In [6]:
# Instala langchain
#!pip install langchain
#!pip install langchain-core
#!pip install langchain-community
#!pip install scikit-learn
#!!pip install faiss-cpu
#!pip install sentence-transformers

In [7]:
# Instala ollama
#!pip install ollama
#!pip install PyMuPDF

In [1]:
# Liberías
import fitz # PyMuPDF para leer el PDF
import numpy as np
import ollama
from sklearn.metrics.pairwise import cosine_similarity

# Respuesta con formato Markdown
from IPython.display import display, Markdown

## 2.Gestión de archivos y contenido

In [3]:
# Listar archivos en una carpeta específica
import os

folder_path = "/Users/cesar/Desktop/chatbot"
files = os.listdir(folder_path)

print("Archivos en la carpeta:")
for file in files:
    print(file)

Archivos en la carpeta:
.DS_Store
Camino de Servidumbre.pdf
__init__.py
__pycache__
utils.py
app.py
Stanford Jobs.pdf
Pasaporte a Magonia.pdf
chatbot_llama2 semantico interfaz 1.0.ipynb
De Cero a Uno.pdf


In [4]:
# Leer el archivo PDF desde el directorio
pdf_path = os.path.join(folder_path, "Pasaporte a Magonia.pdf")  # Ajusta la ruta al archivo


In [5]:
# Función que extrae texto y lo pasa a formato string
def extract_text_from_pdf(pdf_path):
    
    # Abre el archivo PDF
    doc = fitz.open(pdf_path)
    full_text = ""

    # Extrae texto de cada página del PDF
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        full_text += page.get_text("text")

    return full_text

In [12]:
# Extraer texto del PDF
pdf_content = extract_text_from_pdf(pdf_path)

In [13]:
# Una vista del contenido
display(Markdown(pdf_content[:1500]))

Camino de servidumbre: Textos y
documentos, Friedrich A. Hayek 
La Biblioteca de la Libertad busca poner a disposición del público de habla
hispana, de manera gratuita, libros clásicos relacionados a la filosofía liberal. Este
es un proyecto elaborado en conjunto por ElCato.org y Liberty Fund, Inc., que
coinciden en su misión de promover las ideas sobre las que se fundamenta una
sociedad libre.
   
Los libros que se encuentran en la Biblioteca comprenden una amplia gama de
disciplinas, incluyendo economía, derecho, historia, filosofía y teoría política. Los
libros están presentados en una variedad de formatos: facsímile o PDF de imágenes
escaneadas del libro original, HTML y HTML por capítulo y PDF de libro
electrónico.
   
Sobre el autor
   
Friedrich August von Hayek (1899 – 1992) nació en
Viena, Austria, que en ese entonces era una de las grandes
capitales intelectuales de Europa. Hayek es particularmente
conocido como un defensor del liberalismo clásico y del
capitalismo en contra del socialismo y el pensamiento
colectivista. Fue miembro de la Escuela Austriaca de
economía y escribió extensamente acerca teoría monetaria,
el cálculo en una economía socialista, la teoría de los
órdenes espontáneos y la teoría del derecho evolutivo.
Inició su carrera como profesor universitario en la
Universidad de Viena, luego en la London School of Economics y posteriormente
en la Universidad de Chicago y en la Universidad de Freiburg. En 1974 obtuvo el
Premio Nobel de Economía por su trab

In [14]:
# Función que limpia el texto
import re

def clean_text(document):
    # 1. Reemplaza saltos de línea por espacios
    text = document.replace("\n\n", " ")

    # 2. Normaliza caracteres especiales (opcionalmente usa `unidecode` para mayor limpieza)
    text = re.sub(r"\\ń", "́", text)  # Corrige '\ń'
    text = re.sub(r"\\n", " ", text)  # Corrige '\n'

    # 3. Reemplaza múltiples espacios consecutivos por un único espacio
    text = re.sub(r"\s+", " ", text)

    # 4. Opcional: Elimina texto no deseado como números de página o encabezados
    text = re.sub(r"\d+\s?\.?S T E V E J O B S.*", "", text)  # Elimina encabezados específicos

    # 5. Normaliza espacios alrededor de signos de puntuación
    text = re.sub(r"\s+([.,!?])", r"\1", text)

    # 6. Elimina espacios iniciales y finales
    text = text.strip()

    return text

In [15]:
# Limpiar el texto
pdf_content_clean = clean_text(pdf_content)

In [16]:
# Una vista del contenido limpio
display(Markdown(pdf_content_clean[:1500]))

Camino de servidumbre: Textos y documentos, Friedrich A. Hayek La Biblioteca de la Libertad busca poner a disposición del público de habla hispana, de manera gratuita, libros clásicos relacionados a la filosofía liberal. Este es un proyecto elaborado en conjunto por ElCato.org y Liberty Fund, Inc., que coinciden en su misión de promover las ideas sobre las que se fundamenta una sociedad libre. Los libros que se encuentran en la Biblioteca comprenden una amplia gama de disciplinas, incluyendo economía, derecho, historia, filosofía y teoría política. Los libros están presentados en una variedad de formatos: facsímile o PDF de imágenes escaneadas del libro original, HTML y HTML por capítulo y PDF de libro electrónico. Sobre el autor Friedrich August von Hayek (1899 – 1992) nació en Viena, Austria, que en ese entonces era una de las grandes capitales intelectuales de Europa. Hayek es particularmente conocido como un defensor del liberalismo clásico y del capitalismo en contra del socialismo y el pensamiento colectivista. Fue miembro de la Escuela Austriaca de economía y escribió extensamente acerca teoría monetaria, el cálculo en una economía socialista, la teoría de los órdenes espontáneos y la teoría del derecho evolutivo. Inició su carrera como profesor universitario en la Universidad de Viena, luego en la London School of Economics y posteriormente en la Universidad de Chicago y en la Universidad de Freiburg. En 1974 obtuvo el Premio Nobel de Economía por su trabajo relaciona

## 3.Generación de chunks

### Caso 1: Genera los chunks de texto a partir de una cantidad predefinida de caracteres y de solapamiento.

In [17]:
# Caso 1: Función que genera los chunks de texto por cantidad de caracteres y solapamiento
import nltk
from langchain.text_splitter import NLTKTextSplitter

# Inicializar el tokenizador de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')

def split_text_nltk(text, chunk_size=500, overlap=50):
    text_splitter = NLTKTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter.split_text(text)

[nltk_data] Downloading package punkt to /Users/cesar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/cesar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
# Dividir el texto en chunks
split_chunks = split_text_nltk(pdf_content_clean)
print(len(split_chunks))

Created a chunk of size 575, which is longer than the specified 500
Created a chunk of size 663, which is longer than the specified 500
Created a chunk of size 512, which is longer than the specified 500
Created a chunk of size 692, which is longer than the specified 500
Created a chunk of size 650, which is longer than the specified 500
Created a chunk of size 876, which is longer than the specified 500
Created a chunk of size 508, which is longer than the specified 500
Created a chunk of size 522, which is longer than the specified 500
Created a chunk of size 572, which is longer than the specified 500
Created a chunk of size 1272, which is longer than the specified 500
Created a chunk of size 559, which is longer than the specified 500
Created a chunk of size 520, which is longer than the specified 500
Created a chunk of size 622, which is longer than the specified 500
Created a chunk of size 810, which is longer than the specified 500
Created a chunk of size 599, which is longer th

2180


In [19]:
# Imprime los 10 primeros chunks (o uno de ellos)
display(Markdown(split_chunks[5]))

Para Hayek, las ideas colectivistas —ya sean de izquierda o de derecha — no conducen a una utopía sino que al darle cada vez más poder al Estado para controlar la economía, inevitablemente conducen a horrores como los de la Alemania Nazi y la Italia Fascista.

Liberty Fund es una fundación educativa privada establecida para fomentar el estudio del ideal de una sociedad de individuos libres y responsables.

### Caso 2: Genera chunks semánticos
- Semantic Chunk crea embeddings de oraciones individuales para ofrecer trozos "chunks" más significativos.
- Chunks pequeños mantienen mejor el significado y la relevancia y los chunks largos mejoran el contexto.
- Dado un documento, se divide por frases y se agrupan. Luego valida la similitud dentro de un valor umbral y se agrupan para obtener trozos más grandes. Si están fuera del umbral se mantienen separadas.
- Este paquete se encuentra en Langchain experimental (al momento de desarrollar este código) y se instaló usando conda para reducir la posibilidad de conflictos con otras dependencias del ambiente.  

- Métodos de umbral:
    - Percentil.
    - Desviación estándar.
    - Intercuartil.

- Pasos del proceso:  

    - División del documento: Divide el corpus de texto en frases con base a signos de puntuación (ej.: .,?,y !).  
    - Indexación de frases: Asigna un índice a cada frase según su posición en el cuerpo de texto.  
    - Buffering: Añade un  __"buffer_size"__ (int) de frases alrededor de cada sentencia clave para contextualizar.  
    - Cálculo de distancias: Mide las distancias semánticas entre los grupos de frases.  
    - Unión basada en similitud: Fusiona los grupos de oraciones que son similares utilizando como referencia los límites establecidos por los métodos de chunking.

In [20]:
# Caso 2: Función que genera los chunks de texto semánticos
from langchain_experimental.text_splitter import SemanticChunker     # Chunks semántico experimental
from langchain.embeddings import HuggingFaceBgeEmbeddings            # modelo de embeddings

semantic_chunker = SemanticChunker(HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"), 
breakpoint_threshold_type="percentile")

/opt/anaconda3/envs/llm_chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Generar los chunks semánticos
semantic_chunks = semantic_chunker.create_documents([pdf_content_clean])

In [22]:
# Imprime los 10 primeros chunks (o uno de ellos)
display(semantic_chunks)

[Document(metadata={}, page_content='Camino de servidumbre: Textos y documentos, Friedrich A. Hayek La Biblioteca de la Libertad busca poner a disposición del público de habla hispana, de manera gratuita, libros clásicos relacionados a la filosofía liberal. Este es un proyecto elaborado en conjunto por ElCato.org y Liberty Fund, Inc., que coinciden en su misión de promover las ideas sobre las que se fundamenta una sociedad libre. Los libros que se encuentran en la Biblioteca comprenden una amplia gama de disciplinas, incluyendo economía, derecho, historia, filosofía y teoría política. Los libros están presentados en una variedad de formatos: facsímile o PDF de imágenes escaneadas del libro original, HTML y HTML por capítulo y PDF de libro electrónico. Sobre el autor Friedrich August von Hayek (1899 – 1992) nació en Viena, Austria, que en ese entonces era una de las grandes capitales intelectuales de Europa. Hayek es particularmente conocido como un defensor del liberalismo clásico y de

## 4.Embeddings y vector store

### Caso 1: Split chunks

- Genera los embeddings y el vector store para los split chunks

In [23]:
# Función que genera los embeddings y construye el vector store (base vectorial)
from langchain.embeddings import HuggingFaceBgeEmbeddings    # modelo de embeddings
from langchain.vectorstores import FAISS                     # vector stores

def create_vector_store(texts):
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # Usa un modelo para generar los embeddings
  vector_store = FAISS.from_texts(texts, embeddings)
  return vector_store

In [24]:
# Crear el vector store con embeddings para split
split_vector_store = create_vector_store(split_chunks)

### Caso 2: Semantics chunks

- Genera el vector store para los chunks semánticos

In [25]:
# Crear el vector store con embeddings semantic
from langchain.vectorstores import FAISS                     # vector stores

semantic_chunks_vector_store = FAISS.from_documents(semantic_chunks, 
embedding=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

In [26]:
# Considera un documento de texto
semantic_chunk_retriever = semantic_chunks_vector_store.as_retriever(search_kwargs={"k":1})

In [27]:
semantic_chunk_retriever.invoke("¿Qué es la libertad?")

[Document(metadata={}, page_content='—Ed.] [257] Mr. Peter Drucker (The End of Economic Man, cit., 74) observa correctamente que «cuanto menos libertad hay, más se habla de la “nueva libertad”. Pero esta nueva libertad es una simple palabra que cubre exactamente lo contrario de todo lo que Europa entendió siempre por libertad... La nueva libertad que se predica en Europa es, en realidad, el derecho de la mayoría contra el individuo». [La primera parte de la cita aparece en realidad en la página 79, y la última parte en la página 80. —Ed.] [258] Sidney y Beatrice Webb, Soviet Communism: A New Civilization?, cit., Biblioteca de la Libertad: Camino de servidumbre 256 vol. 2, p.')]

## 5.Modelo LLM

### Nota: 

- Aunque use la API para enviar solicitudes de generación de texto a través de un puerto (como un servicio remoto), el modelo Llama2 tiene que estar físicamente en la máquina para que el servidor de Ollama pueda acceder a él y procesar esas solicitudes. El comando ollama pull llama2 descarga e instala el modelo en la máquina, permitiendo que el servidor de Ollama funcione correctamente y responda a las solicitudes de la API. La API permite acceder al modelo desde el código, pero el modelo debe estar presente localmente en la máquina para que el servidor pueda ejecutarlo y dar respuestas.

    1.- Instalación del modelo: Al usar el comando ollama pull llama2, descarga el modelo Llama2 y lo guarda en la computadora.  
    2.- Ejecución del servidor local: Cuando ejecuta ollama start, inicia un servidor local en la máquina que está "escuchando" en un puerto específico (en este caso, el puerto es el 11434). Este servidor es un proceso que maneja las solicitudes (como la que se hace desde el código con LangChain) y genera respuestas usando el modelo descargado.  
    3.- Consulta local: Cuando desde el código en Python se usa llm.invoke(prompt) o response = llm(prompt) en LangChain, la consulta no se va a internet ni a servidores externos. La consulta va directamente a este servidor local que está corriendo en la máquina. El servidor toma la solicitud, pasa el prompt al modelo Llama2, y el modelo genera una respuesta que es enviada de vuelta al código.  
    4.- No se envía nada fuera de la computadora: No se está "simulando" nada, pero sí está usando una simulación de servidor local que permite a la máquina actuar como si estuviera interactuando con un servidor remoto, pero en realidad todo ocurre en el equipo.  

In [28]:
# Habilitar el modelo
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama2",               # Nombre del modelo a utilizar
    temperature=1.0,              # Aleatoriedad de las respuestas (1.0 por defecto)
    top_p=1.0                     # Probabilidad acumulada para limitar diversidad (1.0 por defecto)
)

"""llama2""";

"""llama2:7b""";

/var/folders/19/4tjx9v5x7wz32t2mq2yk7sv80000gn/T/ipykernel_6885/247208326.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


## 6.Interacción

In [29]:
# Ingresar una pregunta

# De Cero a Uno
# query = "¿Cuál es la opinión del autor sobre los monopolios en el contexto de la industria tecnológica?"
# query = "¿Qué dice el autor acerca de su experiencia de dirigir Paypal?"
# query = "¿Qué piensa el autor sobre ser el primero en mover ficha o ser el primero en entrar a un mercado?"

# Steve Jobs
# query = "¿Cuál es la idea que tiene el autor sobre su enfermedad y la angustia de enfrentar a la muerte?"

# Camino de servidumbre
query = "¿Por qué el autor piensa que 'los peores' se ponen siempre a la cabeza?"


In [30]:
# Generar el embedding de la pregunta
embedding_query = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2").embed_query(query)

### Caso 1: Respuesta para split chunk

In [31]:
# Buscar los textos más similares con la pregunta dentro el vector store split
similar_docs_splits = split_vector_store.similarity_search_by_vector(embedding_query, k=3)  # k=3 devuelve las 3 más similares

In [32]:
# Una vista de los textos más similares split
similar_docs_splits

[Document(metadata={}, page_content='Este punto de vista requiere solamente emoción y autoestima —fáciles de conseguir y también satisfactorias—.\n\nPara entender por qué los hombres “buenos” en posición de poder producirán el mal, mientras que el hombre ordinario sin poder pero que pueda comprometerse en la cooperación voluntaria con sus vecinos producirá el bien, requiere análisis y reflexión, subordinando las emociones a las facultades racionales.'),
 Document(metadata={}, page_content='Mas el paro o la pérdida de renta a que siempre se verá sometido alguien en cualquier sociedad es, sin duda, menos degradante si resulta de la mala suerte y no ha sido impuesto deliberadamente por la autoridad.\n\nPor amargo que sea el trance, lo sería mucho más en una sociedad planificada.\n\nEn ella, alguien tendría que decidir no sólo si una persona es necesaria en una determinada ocupación, sino incluso si es útil para algo y hasta qué punto lo es.'),
 Document(metadata={}, page_content='El probl

In [33]:
# Extraer los dos textos más relevantes basados en la similaridad split
context_split = " ".join([doc.page_content for doc in similar_docs_splits[:2]])

In [34]:
# Prompt split
prompt_split = f"""
A continuación, se presenta un contexto para responder la pregunta. Considera cuidadosamente la información de este contexto\n
y responde en español, de forma clara, precisa y en un tono accesible y cercano, cuidando de no truncar el texto de tu respuesta.

Contexto:
{context_split}

Pregunta: {query}

Respuesta (en español en un tono claro, preciso, accesible, cercano y sin truncar):
"""

In [35]:
# Una vista del prompt que combina contexto y pregunta split
display(Markdown(prompt_split))


A continuación, se presenta un contexto para responder la pregunta. Considera cuidadosamente la información de este contexto

y responde en español, de forma clara, precisa y en un tono accesible y cercano, cuidando de no truncar el texto de tu respuesta.

Contexto:
Este punto de vista requiere solamente emoción y autoestima —fáciles de conseguir y también satisfactorias—.

Para entender por qué los hombres “buenos” en posición de poder producirán el mal, mientras que el hombre ordinario sin poder pero que pueda comprometerse en la cooperación voluntaria con sus vecinos producirá el bien, requiere análisis y reflexión, subordinando las emociones a las facultades racionales. Mas el paro o la pérdida de renta a que siempre se verá sometido alguien en cualquier sociedad es, sin duda, menos degradante si resulta de la mala suerte y no ha sido impuesto deliberadamente por la autoridad.

Por amargo que sea el trance, lo sería mucho más en una sociedad planificada.

En ella, alguien tendría que decidir no sólo si una persona es necesaria en una determinada ocupación, sino incluso si es útil para algo y hasta qué punto lo es.

Pregunta: ¿Por qué el autor piensa que 'los peores' se ponen siempre a la cabeza?

Respuesta (en español en un tono claro, preciso, accesible, cercano y sin truncar):


In [36]:
# Usar el modelo para generar la respuesta split
response_split = llm(prompt_split)

/var/folders/19/4tjx9v5x7wz32t2mq2yk7sv80000gn/T/ipykernel_6885/1874413963.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_split = llm(prompt_split)


In [37]:
# Respuesta con formato split
display(Markdown((clean_text(response_split))))

El autor de esta obra piensa que "los peores" se ponen siempre a la cabeza porque, en su opinión, la sociedad está diseñada de manera que los individuos con más poder y influencia son los que más fácilmente pueden causar daños a la sociedad. Esto puede deberse a varios factores, como la desigualdad económica, la explotación laboral o simplemente la falta de control sobre el uso del poder y la influencia en la toma de decisiones. En una sociedad planificada, donde la autoridad tiene un mayor control sobre la distribución de recursos y el estatus social, es más probable que los individuos con más poder y influencia se conviertan en los responsables del mal en la sociedad. En este escenario, la mala suerte o la pérdida de renta no serían tan dañinas para una persona ya que habría medidas de seguridad social y sistemas de apoyo para ayudarla a superar esos moments difíciles. Sin embargo, en una sociedad capitalista o en la que no existe un sistema de seguridad social, la mala suerte o la pérdida de renta pueden tener efectos devastadores en una persona y sus cercanos. En este caso, la autoridad puede ser más lenta o ineficiente en brindar ayuda, lo que podría empeorar la situación de la persona afectada. En resumen, el autor piensa que "los peores" se ponen siempre a la cabeza debido a la desigualdad y la falta de control sobre el poder y la influencia en una sociedad.

### Caso 2: Respuesta para semantic chunk

In [38]:
# Buscar los textos más similares con la pregunta dentro el vector store semántico
similar_docs_semantic = semantic_chunks_vector_store.similarity_search_by_vector(embedding_query, k=3)  # k=3 devuelve las 3 más similares

In [39]:
# Una vista de los textos más similares semánticos
similar_docs_semantic

[Document(metadata={}, page_content='Hayek, es una magistral demostración del trabajo que se propone desarrollar. Este trabajo consiste en mostrar por medio de un razonamiento general e histórico, y esto último sobre todo con referencia al curso de los acontecimientos en Alemania, dos cosas: primero, que cualquier política que se diga socialista, o de economía planificada, conduce inevitablemente al totalitarismo y a la dictadura; y, segundo, que un orden social así acaba cayendo inevitablemente bajo el control de «los peores» individuos. Naturalmente, el argumento es político más que económico, excepto en el sentido indirecto de que los problemas a resolver y las funciones a realizar por el sistema de organización de mercado abierto son económicos y que no pueden ser llevados a cabo por un gobierno en un orden político libre, ni por el propio sistema de mercado libre bajo un régimen político democrático. Hay poca o ninguna teoría económica en el libro. Los quince breves capítulos desc

In [40]:
# Extraer los dos textos más relevantes basados en la similaridad semántica
context_semantic = " ".join([doc.page_content for doc in similar_docs_semantic[:2]])

In [41]:
# Prompt semántico
prompt_semantic = f"""
A continuación, se presenta un contexto para responder la pregunta. Considera cuidadosamente la información de este contexto\n
y responde en español, de forma clara, precisa y en un tono accesible y cercano, cuidando de no truncar el texto de tu respuesta.

Contexto:
{context_semantic}

Pregunta: {query}

Respuesta (en español en un tono claro, preciso, accesible, cercano y sin truncar):
"""

In [42]:
# Una vista del prompt que combina contexto y pregunta
display(Markdown(prompt_semantic))


A continuación, se presenta un contexto para responder la pregunta. Considera cuidadosamente la información de este contexto

y responde en español, de forma clara, precisa y en un tono accesible y cercano, cuidando de no truncar el texto de tu respuesta.

Contexto:
Hayek, es una magistral demostración del trabajo que se propone desarrollar. Este trabajo consiste en mostrar por medio de un razonamiento general e histórico, y esto último sobre todo con referencia al curso de los acontecimientos en Alemania, dos cosas: primero, que cualquier política que se diga socialista, o de economía planificada, conduce inevitablemente al totalitarismo y a la dictadura; y, segundo, que un orden social así acaba cayendo inevitablemente bajo el control de «los peores» individuos. Naturalmente, el argumento es político más que económico, excepto en el sentido indirecto de que los problemas a resolver y las funciones a realizar por el sistema de organización de mercado abierto son económicos y que no pueden ser llevados a cabo por un gobierno en un orden político libre, ni por el propio sistema de mercado libre bajo un régimen político democrático. Hay poca o ninguna teoría económica en el libro. Los quince breves capítulos describen hábilmente el viejo liberalismo y lo comparan con las tendencias actuales, que son virtualmente antitéticas y discuten problemas tales como el individualismo, la democracia, el estado de derecho, seguridad y libertad, el lugar de la verdad en la vida política y social, la relación entre las condiciones materiales y los fines ideales, y el problema del orden internacional. Cuando digo que el argumento está bien desarrollado, es sintético y concluyente, debería añadir que la postura que defiende coincide con mi convicción anterior a la lectura de este trabajo. Pueden hallarse opiniones notablemente inteligentes contra este punto de vista y estaría bien obtener un informe de alguien que mantenga esta postura contraria. Una persona así podemos encontrarla en esta facultad y en el Departamento de Economía. Desde el punto de vista de que sería deseable la publicación del libro en este país, constato alguna base para la duda. El autor es un refugiado austriaco, un economista muy capaz, que ha sido profesor de la London School of Economics desde mediados los años 1930. Escribe desde un punto de vista claramente inglés, y usa frecuentemente la expresión «este país» refiriéndose a Inglaterra. Aunque trata parcialmente las condiciones de Estados Unidos, y cita escritos estadounidenses, esto es secundario en cuanto a finalidad e intensidad. Este hecho por sí mismo puede limitar el interés en «este país» a un círculo de lectores bastante cultivado, incluso académico. Además, todo el desarrollo muestra un nivel intelectual y universitario más bien alto y la cantidad de conocimiento referente a las condiciones y a la historia de la Europa central es más bien amplio incluso para lectores americanos instruidos. Es difícil verlo como un libro «popular» desde este punto de vista. Biblioteca de la Libertad: Camino de servidumbre 206 Además, hay limitaciones en relación al propio tratamiento, tanto en el argumento teórico como en el histórico. En este último aspecto, el trabajo es esencialmente negativo. Apenas considera el problema de las alternativas, y reconoce inadecuadamente la necesidad y la inevitabilidad política de un amplio espectro de actividades gubernamentales en relación a la vida económica en el futuro. Trata sólo las falacias más simples, peticiones poco razonables y prejuicios románticos que subrayan el clamor popular a favor de un control gubernamental en lugar de la libre empresa. No discute los problemas planteados por las graves carencias de un sistema económico basado en el grado de libertad económica que era considerado deseable y que se permitió, digamos, a caballo de ambos siglos. Y no ataca falacias de una manera dramática, en comparación con el carácter del pensamiento y argumento sobre los que se basan realmente. El tratamiento que hace el autor del curso de los acontecimientos que llevaron a la dictadura nazi en Alemania también me parece que es una notable simplificación. Prácticamente, atribuye todo al movimiento socialista y al paternalismo estatal hacia la clase trabajadora y la industria, incluido el cultivo de una actitud de desprecio hacia la empresa de negocios, en comparación con la estima por el estatus burocrático basado en el salario. Relega explícitamente la tradición militarista a un papel menor.Me parece que hay muchos factores en la historia alemana que deberían ser tenidos en cuenta en un tratamiento equilibrado. Se puede pensar en la tardía supervivencia del feudalismo, que retardó la unificación nacional y la industrialización, y las especiales circunstancias que rodean estos cambios y el establecimiento de un gobierno responsable tras la Primera Guerra mundial. Estos últimos componentes tienen, sin duda, mucho que ver con el fracaso del parlamentarismo, hecho indiscutible y factor vital en el establecimiento del régimen de Hitler.Traigo a colación sólo una breve mención al antisemitismo, que tiene una larga historia en Alemania. Estos asuntos no invalidan, en mi opinión, la conclusión general del autor, pero debilitan el argumento en cuanto presentación de su caso. En suma, el libro es un útil instrumento de trabajo, pero de alcance limitado y algo parcial en su tratamiento. No estoy seguro de que vaya a tener un amplio mercado en este país, o de que pudiese llegar a cambiar la postura de muchos lectores. Informe de Jacob Marschak[391] 20 de diciembre de 1943 La habitual discusión entre partidarios y adversarios de la libre empresa no se ha desarrollado a un nivel muy alto hasta hoy. El libro de Hayek puede dar comienzo en este país a un tipo de debate más académico. El libro se dirige a los amigos de la libre empresa y les proporciona nuevos materiales: la interpretación de Hayek de la actual escena inglesa (los trabajadores y Biblioteca de la Libertad: Camino de servidumbre 207 los monopolios industriales que van juntos hacia la economía colectiva) resultará nueva para todos los lectores americanos excepto para aquellos que han leído o escuchado las opiniones de William Benton; mientras que el trasfondo alemán de Hayek lo capacita para dar nuevo apoyo al debate respecto a que el socialismo es el padre del nazismo. Quienes no están convencidos de entrada de las tesis de Hayek probablemente aprenderán más de sus argumentos que los que sí lo están. Hayek (Capítulo IV) siente un gran desprecio por el método cuasi-científico de las «tendencias», de las «las oleadas del mañana». Los partidarios de la planificación porque aman la voluntad inevitable, quizá tras leer a Hayek revisen su fe o sus gustos. Quizá comiencen a pensar en términos de fines y medios y no de profecías. Es cierto que el propio Hayek alimenta muy poco este pensamiento concreto. Como él mismo dice al final del libro (páginas 177, 179),[392] este es casi exclusivamente crítico, no constructivo. Su técnica es de blanco y negro. Se muestra impaciente por llegar a compromisos (página 31). Está escrito con la pasión y la ardiente claridad de un gran doctrinario.Hayek tiene la sinceridad de alguien que ha tenido una visión del peligro que otros no han visto. Advierte a sus semejantes con amorosa impaciencia. Así, pues, los mejores capítulos del libro son negativos o formales.Hay un excelente y realmente inspirado capítulo, el «Estado de derecho» (Capítulo VI); pero Hayek tiene poco que decir en cuanto a cómo el Estado de Derecho (es decir, la evitación de las decisiones administrativas ad hoc) podrían aplicarse como instrumentos para mitigar el desempleo por medios monetarios, o para combatir a los monopolistas. Sobre tales puntos Hayek da sólo vagas pistas (páginas 90, 147).Ya que en este país los términos «plan» y «socialismo» se han utilizado frecuentemente en el sentido de incluir las políticas monetarias y fiscales, la seguridad social, e incluso el impuesto progresivo sobre la renta, el lector americano esperará posiblemente que Hayek haga una demarcación concreta entre lo que el libro llama «planificación en el buen sentido» y la (no deseable) planificación como tal. De hecho, los capítulos no económicos (el que trata de «El fin de la verdad», por ejemplo) son más imponentes que los económicos. Los que leen a Walter Lippmann, a Stuart Chase[393], o la discusión de Fortune sobre el mundo de posguerra leerá también a Hayek. Suele ser menos concreto que Lippmann o que Chase; pero su pensamiento es algo más incisivo, precisamente porque es más abstracto. El estilo de Hayek es ameno y ocasionalmente inspirado. No debemos ignorar este libro. J.MARSCHAK Prólogo de John Chamberlain a la edición americana Biblioteca de la Libertad: Camino de servidumbre 208 Los slogans de nuestro tiempo se expresan con una variedad de términos: «pleno empleo», «planificación», «seguridad social», «liberación de la escasez». La realidad de nuestro tiempo sugiere que ninguna de estas cosas deben mantenerse cuando se convierten en objetos conscientes de la política gubernamental. Son palabras falsas. En Italia corrompen a un pueblo y lo conducen a la muerte bajo el ardiente sol africano. En Rusia se dio el primer Plan Quinquenal; y se dio también la liquidación de tres millones de kulaks. En Alemania hubo pleno empleo entre 1935 y 1939; pero 600.000 judíos ya han sido privados de sus propiedades, dispersados a los confines de la tierra, o yacen en fosas comunes en los bosques polacos.Y en Estados Unidos la bomba nunca se llenó tras los sucesivos intentos; sólo la guerra salvó a los políticos del «pleno empleo». Hoy sólo un puñado de autores han osado establecer una conexión entre nuestros slogans y el terror que acosa al mundo moderno. Entre estos autores se halla F.A. Hayek, economista austriaco que ahora vive en Inglaterra. Habiendo presenciado el congelamiento de los sistemas sociales y económicos alemán, italiano y de los países danubianos, Hayek está horrorizado al ver cómo los ingleses sucumben gradualmente antes las ideas de economía controlada del alemán Walter Rathenau, los sindicalistas italianos —y, sí, de Adolf Hitler que tiene el valor de trazar conclusiones del menos rotundo estatismo de sus predecesores.[394] Este libro de Hayek —The Road to Serfdom— es una advertencia, un grito en un tiempo de duda. Y dice a los británicos y, por implicación, a los americanos: Alto, mirad y escuchad. The Road to Serfdom es sobrio, lógico, severo. No trata de congraciarse con el lector.Pero la lógica es incontestable: «pleno empleo», «seguridad social» y «liberación de la escasez» no se tendrán a menos que sea como sucedáneos de un sistema que libere las energías de los individuos. Cuando «la sociedad» y el «bien de todos» y «el mayor bien para el mayor número» se convierten en la piedra de toque dominante de la acción del estado, ningún individuo puede planear su propia existencia. Pues los «planificadores» estatales deben arrogarse a sí mismos el derecho de entrar en cada sector del sistema económico si el bien de la «sociedad» o el «bienestar general» están por encima de todo. Si los derechos del individuo se interponen, hay que marginarlos. La amenaza del «dinamismo» del estado desemboca en un amplio temor, por lo general inconsciente, entre todos los intereses productivos que todavía conservan una libertad de acción condicional. Y el temor afecta a los resortes de la acción. La gente puede tratar de ser más lista que el gobierno lo mismo que ayer trataba de engañar al mercado. Pero hay esta diferencia: los factores del mercado obedecían al menos leyes relativamente objetivas, mientras que los gobiernos están notablemente sujetos al capricho. Uno puede arriesgar su futuro por un juicio que cuenta con stocks, puntos de saturación del mercado, tasas de interés, curvas de tendencia de los deseos de los compradores. Pero ¿cómo puede un individuo engañar a un Biblioteca de la Libertad: Camino de servidumbre 209 gobierno cuya meta es suspender las leyes objetivas del mercado cuando y donde quiere hacerlo en nombre de la «planificación»? Con perspicacia, Peter Drucker observaba una vez que los «planificadores» son todos improvisadores[395]. Éstos no crean certeza, sino incertidumbre para los individuos.Y,como Hayek demuestra, el resultado final de la incertidumbre es la guerra civil, o la dictadura que evita la guerra civil. La alternativa a la «planificación» es el «estado de derecho». Hayek no es incondicional del laissez-faire; cree en un plan para un sistema empresarial. El plan es compatible con niveles de salario mínimo, niveles de salud, la existencia de un mínimo de seguros sociales obligatorios.Y es incluso compatible con ciertos tipos de inversiones gubernamentales. Pero la cuestión es que el individuo debe saber, con antelación, cómo van a funcionar las reglas. No puede planear su negocio, su futuro, incluso sus propios asuntos familiares, si el «dinamismo» de una autoridad planificadora central pende sobre su cabeza. En ciertos aspectos, Hayek es más «inglés» que los ingleses de hoy. Pertenece, con modificaciones, al gran linaje de Manchester, no a la escuela de los Webbs[396]. Puede ser que sea incluso más «americano» que los americanos actuales. Si es así, sólo podemos augurar una acogida en Estados Unidos lo más amplia posible de The Road to Serfdom. Carta de John Scoon a C. Hartley Grattan[397] Hayek: The Road to Serfdom 2 de mayo de 1945 Sr. C. Hartley Grattan 6 White Hall Road Tuckahoe, Nueva York Estimado Sr. Grattan: He estado en mi despacho unos cinco minutos al día desde que volví de Chicago, pues, de lo contrario, habría sabido usted de mí antes. 19. —Ed.] [352] Otro elemento que después de esta guerra reforzará probablemente las tendencias en esta dirección lo constituirán algunos de los hombres que durante la guerra saborearon el poder de la intervención coercitiva y encontrarán difícil volver a los humildes papeles que habían desempeñado hasta entonces. Aunque después de la anterior guerra los hombres de esta clase no fueron tan numerosos como lo serán probablemente en el futuro, ejercieron, sin embargo, entonces una influencia no despreciable sobre la política económica de Inglaterra. En la compañía de algunos de estos hombres, hace ya diez o doce años, experimenté por vez primera en este país la entonces inusitada sensación de ser transportado repentinamente a lo que estaba acostumbrado a considerar como una atmósfera intelectual completamente «alemana». [«Diez o doce años» antes Hayek acababa de incorporarse a la facultad en la London School of Economics. Como expongo en mi introducción, pp. 20-21, Hayek difundió la afirmación de que ciertos puntos de vista que entonces eran corrientes entre el público británico recordaban los que existían anteriormente en Alemania en su lección inaugural, «The Trend Economic Thinking,». cit.

Pregunta: ¿Por qué el autor piensa que 'los peores' se ponen siempre a la cabeza?

Respuesta (en español en un tono claro, preciso, accesible, cercano y sin truncar):


In [43]:
# Usar el modelo para generar la respuesta semantic
response_semantic = llm(prompt_semantic)

In [44]:
# Respuesta con formato semantic
display(Markdown((clean_text(response_semantic))))

En el capítulo "The Road to Serfdom" de Friedrich Hayek, el autor argumenta que los seres humanos tienen una tendencia natural a buscar la seguridad y el benéfico del sistema económico, incluso si significa cedérseles cierta cantidad de libertad individual. Esto se debe a que, en su opinión, los seres humanos tienen un temor innato al riesgo y la incertidumbre, y buscan protegerse de estas cosas mediante el establecimiento de una autoridad centralizada con poderes absolutos para planificar y controlar la economía. Hayek cree que esta tendencia se debió a varias causas históricas y culturales. En primer lugar, la humanidad ha pasado por periodos de tiempo en los que la seguridad y la estabilidad económicas eran escasas o no existentes, lo que llevó a un aumento en la demanda de una autoridad centralizada para garantizar la estabilidad y el bienestar. En segundo lugar, la creencia en la existencia de una "planificación" más efectiva ha sido fomentada por la propaganda gubernamental y los medios de comunicación controlados por el estado. El autor también muestra cómo la amenaza del "dinamismo" del estado puede hacer que los intereses productivos se sientan marginados o excluidos del sistema económico, lo que a su vez puede llevar a un aumento en la desigualdad y la discriminación. En resumen, el autor cree que los seres humanos tienen una tendencia natural a buscar la seguridad y el benéfico del sistema económico, incluso si significa cedérseles cierta cantidad de libertad individual, debido a su temor innato al riesgo y la incertidumbre. Esta tendencia puede ser fomentada por una serie de factores históricos y culturales, incluyendo la propaganda gubernamental y los medios de comunicación controlados por el estado.

## LCEL (LangChain Expression Language) RAG Chain   

Recomendado por LangChain  


- Es un sistema declarativo: enfocado en defimir los resultados deseados sin programar explícitamente los pasos para lograrlo.  
- Cadenas computacionales de múltiples pasos: Permiten crear múltiples cadenas y conectarlas en una sola.  
- Permite desarrollar coluciones robustas y escalables.

RunnablePassthrough: Permite pasar la pregunta sin modificaciones.  
semantic_chunk_retriever: Recupera los segmentos semánticos relevantes.  
Pipe operator: Pasa la salida de un componente como la entrada del siguiente, siguiendo el principio declarativo del LCEL.  
StrOutputParser: Analiza la salida del modelo para obtener un resultado en formato de texto.  

Este formato es útil:

1.- Soporte para streaming: Permite manejar casos de uso donde los resultados se generan y envían en tiempo real.  
2.- Soporte asíncrono: Ideal para aplicaciones web que no quieren bloquear el procesamiento.  
3.- Ejecución paralela optimizada: Tareas y cadenas se lanzan en paralelo automáticamente.  
4.- Reintentos y fallbacks: Permite especificar acciones en casos de fallos.  
5.- Facilita la depuración y validación de datos.  
6.- Compatibilidad con LangServe y LangSmith: Fácil creación de APIs y monitorización.  


Fexibilidad y potencia capaz de manejar la mayoría de los casos de uso y facilidad de uso, ya que simplifica la configuración de tareas computacionales complejas.

In [69]:
from langchain_core.runnables import RunnablePassthrough, RunnableMap, RunnableConfig
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

# Crear un prompt
prompt_template = PromptTemplate(
    template="Context: {context_semantic}\nQuestion: {question}\nAnswer:",
    input_variables=["context_semantic", "question"]
)

semantic_rag_chain = (
    RunnableMap(
    {"context_semantic" : semantic_chunk_retriever, 
    "question" : RunnablePassthrough()
    })
    | prompt_template
    | llm
    | StrOutputParser()
)

In [70]:
# El método es capaz de encontrar el documento

# Ejecutar la cadena
result = semantic_rag_chain.invoke(
    "¿Por qué el autor piensa que 'los peores' se ponen siempre a la cabeza?"  # Pregunta
)


In [71]:
# Respuesta
display(Markdown(result))

En "The Road to Serfdom," Friedrich Hayek argues that the worst individuals or groups tend to gain control over society because they are able to manipulate and deceive others to achieve their goals. He suggests that these individuals often use tactics such as propaganda, manipulation of public opinion, and the exploitation of crises to further their interests.

Hayek believes that these "worst" individuals or groups are able to gain power because they are able to tap into the fears and insecurities of others. They use these emotions to create a sense of urgency and crisis, which can then be used to justify the granting of more powers to the state. This, in turn, can lead to a gradual erosion of individual freedom and autonomy as the state becomes increasingly involved in every aspect of society.

Hayek's argument is that once the state gains this level of control, it becomes difficult to reverse the process because the individuals or groups who gained power through manipulation and deceit will not give up their positions easily. They will continue to use their tactics to maintain their power and control over society, leading to a gradual descent into a form of serfdom or totalitarianism.

In short, Hayek believes that the worst individuals or groups are able to gain control over society because they are able to manipulate and deceive others, creating a sense of urgency and crisis that can be used to justify the granting of more powers to the state. This can lead to a gradual erosion of individual freedom and autonomy as the state becomes increasingly involved in every aspect of society.